# Example 5.1: Representation of XP Stabilizer States as Weighted Hypergraph States

This notebook illustrates the algorithms for representing XP codes as Weighted Hypergraph States. 

## Algorithm: Weighted Hypergraph Representation of a Given XP State

**Input:** An XP state $|\phi\rangle =O_{\mathbf{S}_X}|\mathbf{m}\rangle$  of precision $N$. Let $\mathbf{S}_X = \{XP_N(p_i|\mathbf{x}_i|\mathbf{z}_i): 0 \le i < r\}$.

**Output:** A set of generalised controlled phase operators $\{CP(p_i/2N,\mathbf{v}_i)\}$  such that $|\phi\rangle = \Big(\prod_i CP(p_i/2N,\mathbf{v}_i)\Big)\sum_{\mathbf{e}\in \text{ZSupp}(|\phi\rangle)}|\mathbf{e}\rangle$.

**Method**
1. Let $l_i$ be the leading index of $\mathbf{x}_i$ and define the $r \times n$ binary matrix $L$ by setting $L_{ij} = 1$ if $j = l_i$ and $0$ otherwise.
2. Let $\mathbf{p}$ be vector whose entries are indexed by rows $\mathbf{u}\in \mathbb{Z}_2^r$ such that $\mathbf{p}[\mathbf{u}]$ is the phase component of $\mathbf{S}_X^{\mathbf{u}}|\mathbf{m}\rangle$
3. For $\mathbf{u}$ in $\mathbb{Z}_2^r$, ordered by weight then lexicographic order:
    * If $\mathbf{p}[\mathbf{u}] \ne 0$, add the operator $CP(\mathbf{p}[\mathbf{u}]/2N,\mathbf{u}L)$ to the list of operators
    * For all $\mathbf{v} \in \mathbb{Z}_2^r$ such that $\mathbf{v}\mathbf{u} = \mathbf{u}$, set $\mathbf{p}[\mathbf{v}] = (\mathbf{p}[\mathbf{v}]  - \mathbf{p}[\mathbf{u}] ) \mod 2N$


To run different scenarios click here: [![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/m-webster/XPFpackage/HEAD?urlpath=tree/Examples)

This code is Copyright 2021-22 Mark Webster, and is made available subject to [GPL licensing](https://www.gnu.org/licenses/gpl-3.0.en.html).

In [1]:
import add_parent_dir
import numpy as np
from common import *
from NSpace import *
from XPAlgebra import *
from XPCodes import *
from whg import *

## Example 5.1: Hypergraph state - Union Jack
# genstr = '''XP_4(0|100011100|011200033)
# XP_4(0|010010010|100100300)
# XP_4(0|001001001|100100300)
# XP_4(0|000100111|211033000)
# XP_4(0|000000000|220020000)
# XP_4(0|000000000|202002000)
# XP_4(0|000000000|200200200)
# XP_4(0|000000000|020200020)
# XP_4(0|000000000|002200002)'''

## Example 5.2: Weighted graph state - universal MBQC
## universal MBQC: non-optimised
genstr = '''XP_4(0|1000111000|1000201000)
XP_4(0|0100100110|0100200100)
XP_4(0|0010010101|0010000102)
XP_4(0|0001001011|0001001002)
XP_4(0|0000000000|2200200000)
XP_4(0|0000000000|2020020000)
XP_4(0|0000000000|2002002000)
XP_4(0|0000000000|0220000200)
XP_4(0|0000000000|0202000020)
XP_4(0|0000000000|0022000002)'''

## universal MBQC: optimised
# genstr = '''XP_4(0|100010|320010)
# XP_4(0|010001|230001)
# XP_4(0|001001|003201)
# XP_4(0|000110|002310)
# XP_4(0|000000|200220)
# XP_4(0|000000|022002)'''

## Code 1
# genstr = 'XP_8(8|0000000|6554444),XP_8(7|1111111|1241234),XP_8(1|1110000|3134444)'

# ## Code 2
# genstr = 'XP_8(0|0000000|1322224),XP_8(12|1111111|1234567)'


G, N = str2XP(genstr)
G = ZMat2D(G)
C = Code(G,N)
S,Em = getVals(C,['S','Em'])

SX, SZ = splitDiag(S)
print('\nSX in canonical form:')
print(XP2Str(SX,N))

Sx = XPx(SX)
RX = makeXP(0,Sx,0) 
# print('\nPhaseless RX such that ORx|0> has the same Z-support as OSx|0>:')
# print(XP2Str(RX,N))
L = leadingIndices(Sx)
m,n = np.shape(Sx)
L = ZMat([set2Bin(n,[l]) for l in L])
print("\nLeading index matrix L =")
print(ZmatPrint(L))

OSx, ix = OrbitOperator(SX,N)
ORx, ix = OrbitOperator(RX,N)

for m in Em:
    print('\nCaluclate phase function: m =',ZMat2str(m))
    Xm = makeXP(0,m,0)
    phi = XPMul(OSx,Xm,N)
    print('phi = OSx|m> =')
    print(State2Str(phi,N))

    CPList = phaseFunction(SX,m,N)
    CPemb = CPEmbed(CPList,Sx)
    print('Phase Function f(u) of phi:')
    print(CP2Str(CPemb))

    ## apply CPList to ORx|m>
    psi = XPMul(ORx,Xm,N)
    psi = CPApply(CPemb,psi,N)
    print('Phase function check:', StateEqual(phi,psi))


SX in canonical form:
XP_4(0|1000111000|1000003020)
XP_4(0|0100100110|0100002120)
XP_4(0|0010010101|0010000102)
XP_4(0|0001001011|0001001002)

Leading index matrix L =
1000000000
0100000000
0010000000
0001000000

Caluclate phase function: m = 0000000000
phi = OSx|m> =
|0000000000>+|0001001011>+|0010010101>+w4/8|0011011110>+|0100100110>+|0101101101>+w2/8|0110110011>+w6/8|0111111000>+|1000111000>+w2/8|1001110011>+|1010101101>+w6/8|1011100110>+w4/8|1100011110>+w6/8|1101010101>+w6/8|1110001011>+w4/8|1111000000>
Phase Function f(u) of phi:
CP(1/2,1100000000)
CP(1/4,0110000000)
CP(1/4,1001000000)
CP(1/2,0011000000)
Phase function check: True
